In [3]:
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb

# import credentials
# from config.secrets import *

In [7]:
# create client

key = FULCRUM_CRED.get("api_key")

# test: Query a form from fulcrum (Preventive maintenance)
fulcrum = fc.Fulcrum(key = key)

# print(fulcrum)
# forms = fulcrum.forms.search(url_params={'id': '44359e32-1a7f-41bd-b53e-3ebc039bd21a'})
# print(forms)

form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
form = fulcrum.forms.find(form_id)

print(form)

{'form': {'name': 'Signal PM', 'description': None, 'version': 109, 'bounding_box': [30.266033, -97.8995454, 30.4979097, -97.6793988], 'record_title_key': '6080', 'title_field_keys': ['6080'], 'status_field': {'type': 'StatusField', 'label': 'Status', 'key': '@status', 'data_name': 'status', 'default_value': 'incomplete', 'enabled': False, 'read_only': True, 'hidden': False, 'description': '', 'choices': [{'label': 'Not Complete', 'value': 'incomplete', 'color': '#CB0D0C'}, {'label': 'Completed', 'value': 'completed', 'color': '#2D5D00'}], 'required': False, 'disabled': False, 'default_previous_value': False}, 'auto_assign': False, 'hidden_on_dashboard': False, 'record_count': 40, 'geometry_types': ['Point'], 'geometry_required': False, 'script': "function setStatus() {\n  SETSTATUS('completed')\n}\n\n// config\nvar signals = { \n    'id' : 'xwqn-2f78',\n    'label_field' : 'location_name',\n    'value_field' : 'signal_id',\n    'search_distance' : 200,  //  meters\n};\n\n//  use to li

In [8]:
records = fulcrum.records.search(url_params = {"form_id":"44359e32-1a7f-41bd-b53e-3ebc039bd21a"})

# print(len(records['records']))
# print(records['records'][0]['id'])
# records = fulcrum.records.search(url_params={'form_id': 'a1cb3ac7-146f-491a-a4a2-47737fb12074'})
# print(len(records['records']))  # 9
# print(records['records'][0]['id'])  # c90b0edf-0299-42df-bed4-524446d63f40

In [9]:
type(records["records"])

list

In [10]:
print(records["records"][9])

{'status': None, 'version': 2, 'id': '929e506b-b181-4311-bf2e-3a9cd69e2590', 'created_at': '2018-08-13T15:02:09Z', 'updated_at': '2018-08-13T15:19:36Z', 'client_created_at': '2018-08-13T14:57:09Z', 'client_updated_at': '2018-08-13T15:19:32Z', 'created_by': 'Signal Tech', 'created_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'updated_by': 'Signal Tech', 'updated_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'created_location': {'latitude': 30.4664041, 'longitude': -97.8040593, 'altitude': 264.0, 'horizontal_accuracy': 44.265}, 'updated_location': {'latitude': 30.4668103, 'longitude': -97.8046655, 'altitude': 257.0, 'horizontal_accuracy': 12.918}, 'created_duration': 294, 'updated_duration': 16, 'edited_duration': 310, 'form_id': '44359e32-1a7f-41bd-b53e-3ebc039bd21a', 'project_id': None, 'assigned_to': None, 'assigned_to_id': None, 'form_values': {'4de9': {'other_values': [], 'choice_values': ['OK']}, '6080': {'other_values': [], 'choice_values': ['622 | FM 620 RD / PECAN PARK BLVD

In [11]:
PM_dataframe = pd.DataFrame.from_dict(records["records"])

In [12]:
PM_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 28 columns):
altitude               14 non-null float64
assigned_to            0 non-null object
assigned_to_id         0 non-null object
client_created_at      40 non-null object
client_updated_at      40 non-null object
course                 1 non-null float64
created_at             40 non-null object
created_by             40 non-null object
created_by_id          40 non-null object
created_duration       40 non-null int64
created_location       40 non-null object
edited_duration        40 non-null int64
form_id                40 non-null object
form_values            40 non-null object
horizontal_accuracy    40 non-null float64
id                     40 non-null object
latitude               40 non-null float64
longitude              40 non-null float64
project_id             0 non-null object
speed                  0 non-null object
status                 0 non-null object
updated_at        

In [13]:
def recur_dict(col_names, elements):

    for element in elements:
        col_names[element.get("key")] = element.get("data_name")
        for sub_element in element:
            if type(sub_element) == dict:
                recur_dict(col_names, sub_element)
    # for element in elements:
    #     print(type(element))
    #     col_names[element.get("key")] = element.get("data_name")
    #     if type(element) == dict:
    #         recur_dict(col_names, element)
    return col_names

In [14]:
def get_col_names(form_id):
    """Summary
    get a dictionary with {"key": "label"} pair for all columns in a data
    base
    
    
    
    Args:
        form_id (TYPE): Description
    
    
    
    """

    form = fulcrum.forms.find(form_id)
    elements = form.get("form").get("elements")

    col_names = {}

    for element in elements:
    	# column = {element.get("key"): element.get("label")}
    	# columns.append(column)
        col_names[element.get("key")] = element.get("data_name")

    return col_names

In [15]:
def get_records(form_id):
    """Summary
    
    Args:
        form_id (TYPE): Description
    """
    records_dirty = fulcrum.records.search(url_params = {"form_id":form_id})
    
    records_dirty = records_dirty["records"]
    
    
    records = pd.DataFrame.from_dict(records_dirty["form_values"])
    
    return records

In [16]:
def interpret_col_name(records):

    records.rename(columns=col_names)

    return records

In [17]:
def clean_pm(records):
    df = records.copy()

    df["SIGNAL_ID"] = df["signal"].str.split("|").str[0]

    # rename record_id to fulcrum ID
    df = df.rename(columns={'_record_id': 'FULCRUM_ID'})

    df["_server_updated_at"] = df["_server_updated_at"].str.strip("CDT")
    df["PM_COMPLETED_DATETIME"] = pd.to_datetime(df["_server_updated_at"], format = "%Y-%m-%d %H:%M:%S")
    df["PM_COMPLETED_DATE"] = df["PM_COMPLETED_DATETIME"].dt.date

    df["PM_COMPLETED_BY"] = df["technicians"].str.split(",").str[1]
    df["modified_current"] = datetime.now().replace(microsecond=0).isoformat(" ")

    df["modified_current"] =pd.to_datetime(df["modified_current"], format = "%Y-%m-%d %H:%M:%S")

    df["MODIFIED_DATE"] = df["modified_current"] + timedelta(minutes = 20)
    
    df = df[["SIGNAL_ID", "FULCRUM_ID", "PM_COMPLETED_DATE", "MODIFIED_DATE", "PM_COMPLETED_BY"]]

    return df

In [19]:
# start a fulcrum instance
fulcrum = fc.Fulcrum(key = key)
forms = fulcrum.forms.search(url_params={'id': form_id})
# print(forms)
col_names = get_col_names(form_id)
records = get_records(form_id)

# pdb.set_trace()

records = interpret_col_name(records)

# print(col_names)

print(list(records))

TypeError: list indices must be integers or slices, not str